# Molecular Dynamics Simulation of Alanine Dipeptide in Explicit Solvent

In [ ]:
%matplotlib widget

from sys import stdout

import matplotlib.pyplot as plt
import mdtraj
import nglview
import numpy as np
import pandas
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
from simtk.unit import nanometer, picosecond, femtoseconds

In [3]:
pdb = PDBFile('alanine-dipeptide.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open('init3.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in periodic systems.
system = forcefield.createSystem(
    modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy() 
simulation.reporters.append(DCDReporter('traj3.dcd', 100))

simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars3.csv", 100, time=True,
        potentialEnergy=True, totalEnergy=True, temperature=True))

simulation.step(100000)

<Topology; 2 chains, 661 residues, 1996 atoms, 1337 bonds>
#"Step","Temperature (K)","Elapsed Time (s)"
1000,264.81527732508454,9.417533874511719e-05
2000,287.5090468927022,0.23753905296325684
3000,288.4943439860635,0.48711228370666504
4000,289.83621952983214,0.7222506999969482
5000,308.8650889316472,1.0085482597351074
6000,301.51300571954346,1.254230260848999
7000,309.334704862373,1.4963874816894531
8000,303.2849730405985,1.7271511554718018
9000,305.07244766589133,1.9560651779174805
10000,304.578876527116,2.2170662879943848
11000,294.8708109583478,2.447046995162964
12000,289.8033820591522,2.6763904094696045
13000,298.87961205358175,2.902147054672241
14000,298.02058518061875,3.1320972442626953
15000,302.2590732486515,3.362990617752075
16000,305.18383019385124,3.626969814300537
17000,301.1528496062827,3.8594095706939697
18000,294.98869697979995,4.087745666503906
19000,286.81408521548184,4.3201003074646
20000,287.35739781832814,4.550617218017578
21000,297.64677087858206,4.811292886734009

The figure above shows that approximately (the first) 15 picoseconds are required for the equilibration. Results for these steps need to be removed before performing any analysis. A single MD step takes 2 femtoseconds and only every 100 steps, a frame is written to the PDB file, which means that the first 75 frames from the trajectory should be removed. For the visualization, we still look at all steps.

In [ ]:
traj3 = mdtraj.load('traj1.dcd', top='init1.pdb')
view = nglview.show_mdtraj(traj3)
view.clear_representations()
view.add_licorice()
view.add_unitcell()
view

NGLWidget(max_frame=999)